In [1]:
import numpy as np
from scipy import linalg as lin
import nibabel as nib

In [2]:
pwd

'/home/jb/code/pasteur/brainspell-neo/mni-template'

In [3]:
mask_file = "MNI152_T1_3mm_brain_mask_scipy.nii"
mask_img = nib.load(mask_file)
header = mask_img.get_header()

In [4]:
print(header)

<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b'r'
dim_info        : 0
dim             : [ 3 53 63 46  1  1  1  1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : label
datatype        : uint8
bitpix          : 8
slice_start     : 0
pixdim          : [-1.  3.  3.  3.  1.  0.  0.  0.]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 10
cal_max         : 1.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b'spm - 3D normalized'
aux_file        : b''
qform_code      : aligned
sform_code      : aligned
quatern_b       : 0.0
quatern_c       : 1.0
quatern_d       : 0.0
qoffset_x       : 78.0
qoffset_y       : -112.0
qoffset_z       : -50.0
srow_x          : [ -3.   0.  

### dimension of the image

In [5]:
mask_img.shape

(53, 63, 46)

In [6]:
voxel_coo = np.asarray([0,0,0,1])
voxel_coo = voxel_coo[np.newaxis,:].T
print(voxel_coo)

[[0]
 [0]
 [0]
 [1]]


### Coordinates of the (0,0,0)

In [7]:
A = mask_img.get_affine()
talairach_coo = np.dot(A, voxel_coo)
print(talairach_coo)

[[  78.]
 [-112.]
 [ -50.]
 [   1.]]


### Voxel associated with these coordinates

In [14]:
talairach_coo = np.asarray([[-10, 20, 40, 1]]).T
invA = lin.inv(A)
#new_vox_coo = np.rint(invA.dot(talairach_coo))
new_vox_coo = (invA.dot(talairach_coo))
print(new_vox_coo)
print(np.rint(new_vox_coo))

[[ 29.33333333]
 [ 44.        ]
 [ 30.        ]
 [  1.        ]]
[[ 29.]
 [ 44.]
 [ 30.]
 [  1.]]


In [19]:
np.asarray(mask_img.shape).prod()*4

614376